In [13]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt


import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')




train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [14]:
test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


PassengerId: 승객의 고유 번호입니다.
Survived: 승객의 생존 여부를 나타냅니다. 0은 사망, 1은 생존을 의미합니다.
Pclass: 승객의 티켓 등급을 나타냅니다. 1은 1등급(가장 높음), 2는 2등급, 3은 3등급(가장 낮음)을 의미합니다.
Name: 승객의 이름입니다.
Sex: 승객의 성별을 나타냅니다. 'male'은 남성, 'female'은 여성을 의미합니다.
Age: 승객의 나이입니다.
SibSp: 함께 탑승한 형제자매 또는 배우자의 수를 나타냅니다.
Parch: 함께 탑승한 부모 또는 자녀의 수를 나타냅니다.
Ticket: 티켓 번호입니다.
Fare: 승객이 지불한 요금입니다.
Cabin: 승객이 배정받은 객실 번호입니다.
Embarked: 승객이 탑승한 항구를 나타냅니다. C는 Cherbourg, Q는 Queenstown, S는 Southampton을 의미합니다.

In [15]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## 일단 결석치를 전부 없애보자


In [5]:

original_train = train.copy()


full_data = [train, test]


train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
# 일반적으로 pandas에서 누락된 값은 NaN으로 표시되며, 이는 float 타입으로 인식됩니다. 따라서 이 코드는 Cabin 값이 누락되었으면 Has_Cabin을 0으로, 그렇지 않으면 1로 설정합니다.
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)


for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
#dataset.loc[]는 특정 조건을 만족하는 행을 선택하는 데 사용됩니다.
# FamilySize가 1인 승객들에 대해 'IsAlone' 열을 1로 설정합니다.

# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
#fillna('S') 함수는 누락된 값을 ('NaN'이나 'None') 주어진 값으로 대체합니다
#full_data 리스트에 있는 각 데이터셋(train과 test)의 'Embarked' 열의 누락된 값을 'S'로 채웁니다.

for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
#full_data 리스트에 있는 각 데이터셋(train과 test)의 'Fare' 열의 누락된 값을 'train' 데이터셋의 'Fare' 열의 중앙값으로 채웁니다.

# Remove all NULLS in the Age column
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    # 'Age' 열의 표준편차를 계산하고 그 값을 age_std라는 변수에 저장합니다.
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    #np.random.randint() 함수는 주어진 범위에서 정수를 랜덤하게 선택합니다. 여기서는 age_avg - age_std부터 age_avg + age_std까지의 범위를 사용했습니다
    
    # Next line has been improved to avoid warning
    dataset.loc[np.isnan(dataset['Age']), 'Age'] = age_null_random_list
    #np.isnan(dataset['Age'])는 'Age' 열의 각 값이 NaN(누락된 값)인지를 검사하고, 그 결과를 Boolean 형태로 돌려줍니다. 따라서 이 코드는 특정 조건을 만족하는 행을 선택하는 데 사용되는 dataset.loc[]의 조건으로 사용됩니다.
    #그리고 이 조건을 만족하는 행의 'Age' 열에 age_null_random_list의 값을 대입합니다
    dataset['Age'] = dataset['Age'].astype(int)
    #.astype(int) 메소드는 데이터를 원하는 형태로 변환하는데 사용됩니다.
    
# Define function to extract titles from passenger names


    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare']        = 0
    #dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0 이 코드는 'Fare'열의 값이 7.91 이하인 모든 행에 대해 'Fare' 값을 0으로 설정합니다.
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare']      = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age']   = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] ;
    #Age>64인 행들의 'Age' 값을 출력

## title 변수 생성


In [6]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # [A-Za-z]+: 대문자 혹은 소문자로 이루어진 하나 이상의 알파벳 문자열을 나타냅니다. '+' 기호는 하나 이상을 의미합니다.
    #\.: 마침표를 나타냅니다. 마침표는 정규 표현식에서 특수한 의미를 가지므로, 실제 마침표를 찾고자 할 때는 역슬래시(\)로 이스케이프 처리해야 합니다.
    
    if title_search:
        return title_search.group(1)
    return ""
#([A-Za-z]+)\.' 정규표현식에 의해 " Mr. Smith"라는 문자열에서 'Mr'를 찾아냈다면, title_search.group(1)은 이 'Mr'를 반환하게 됩니다.

#만약 찾아낸 그룹이 없다면, title_search는 None이 될 것이고, 이 경우 title_search.group(1)을 호출하면 에러가 발생합니다. 따라서 if title_search: 조건문은 title_search가 None이 아닌, 즉 타이틀이 찾아졌을 때만 title_search.group(1)을 호출하게 하는 역할을 합니다.

for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Name' 열의 각 값에 get_title 함수를 적용하고 그 결과를 새로운 'Title' 열에 저장합니다.
# .apply() 메소드는 DataFrame의 각 행 또는 열에 함수를 적용하는데 사용됩니다.
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
# .replace() 메소드는 DataFrame에서 특정 값을 다른 값으로 대체하는데 사용됩니다.
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    #.map() 메소드는 DataFrame의 각 값에 함수나 딕셔너리를 적용하는데 사용됩니다. 여기서는 딕셔너리 {'female': 0, 'male': 1}를 사용하여 'female'을 0으로, 'male'을 1로 변환합니다.
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Master": 2, "Mrs": 3, "Miss": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [7]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Has_Cabin,FamilySize,IsAlone,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,1,1,0,A/5 21171,0,NaN,0,0,2,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,PC 17599,3,C85,1,1,2,0,3
2,3,1,3,"Heikkinen, Miss. Laina",0,1,0,0,STON/O2. 3101282,1,NaN,0,0,1,1,4


In [8]:
train.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Has_Cabin        0
FamilySize       0
IsAlone          0
Title            0
dtype: int64

In [9]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Has_Cabin        0
FamilySize       0
IsAlone          0
Title            0
dtype: int64

## cabin은 결석치가 너무 많아서 버리겠습니다


In [10]:
for dataset in full_data:
    dataset.drop('Cabin', axis=1, inplace=True)


In [12]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Has_Cabin,FamilySize,IsAlone,Title
0,1,0,3,"Braund, Mr. Owen Harris",1,1,1,0,A/5 21171,0,0,0,2,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,PC 17599,3,1,1,2,0,3
2,3,1,3,"Heikkinen, Miss. Laina",0,1,0,0,STON/O2. 3101282,1,0,0,1,1,4
